In [1]:
import pandas as pd
import altair as alt
import sys
sys.path.append('../')
from mod import load_data, alt_theme

In [2]:
renderer = "jupyterlab"

In [3]:
# Parameters
renderer = "kaggle"


In [4]:
comments, issues, prs = load_data('../../data/')
bot_names = pd.read_csv('../bot_names.csv')
comments = comments.query('author not in @bot_names')
issues = issues.query('author not in @bot_names')
prs = prs.query('author not in @bot_names')

In [5]:
alt.renderers.enable(renderer);
alt.themes.register('my_theme', alt_theme)
alt.themes.enable("my_theme")

ThemeRegistry.enable('my_theme')

In [6]:
author_prs = (
    prs.groupby(['org', 'repo', 'author'])
    .agg({'id': 'count'})
    .rename(columns={'id': 'count'})
    .sort_values(['org', 'repo', 'count'], ascending=False)
)

In [7]:
pony_factors = []
top_authors_all = []
for (org, repo), idata in author_prs.groupby(['org', 'repo']):
    idata = idata.reset_index()
    total_prs = idata['count'].sum()  
    top_authors = idata.groupby('author').sum()['count'].sort_values(ascending=False).to_frame()
    top_authors['org'] = org
    top_cumulative = top_authors['count'].cumsum()
    top_half = top_cumulative > (total_prs / 2)
    pony_factor = (~top_half).sum() + 1
    pony_factors.append((org, repo, pony_factor))
    top_authors_all.append(top_authors.head(20))
pony_factors = pd.DataFrame(pony_factors, columns=['org', 'repo', 'pony'])
top_authors = pd.concat(top_authors_all)

In [8]:
charts = []
for iorg, idata in pony_factors.groupby('org'):
    idata = idata.sort_values(['org', 'pony'], ascending=False)
    ch = alt.Chart(idata, title=f'Organization: {iorg}').mark_bar().encode(
        x=alt.X('repo', sort=idata['repo'].tolist()),
        y='pony',
        color='org'
    )
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:                 org                 repo  pony
    18  binder-examples                    r     3
    2   binder-examples                conda     2
    3   binder-examples     continuous-build     2
    6   binder-examples   dockerfile-rstudio     2
    11  binder-examples       jupyter-stacks     2
    12  binder-examples           jupyterlab     2
    13  binder-examples                latex     2
    15  binder-examples  multi-language-demo     2
    0   binder-examples              appmode     1
    1   binder-examples          apt_install     1
    4   binder-examples           data-quilt     1
    5   binder-examples           demo-julia     1
    7   binder-examples         getting-data     1
    8   binder-examples   getting-data-xroot     1
    9   binder-examples         julia-python     1
    10  binder-examples          jupyter-sos     1
    14  binder-examples   minimal-dockerfile     1
    16  binder-examples                  nix     1
    17  binder-examples     python-conda_pip     1
    19  binder-examples        r_with_python     1
    20  binder-examples         requirements     1
    21  binder-examples             setup.py     1
    22  binder-examples           stencila-r     1,
    encoding: FacetedEncoding({
      color: Color({
        field: 'org',
        type: 'nominal'
      }),
      x: X({
        field: 'repo',
        sort: ['r', 'conda', 'continuous-build', 'dockerfile-rstudio', 'jupyter-stacks', 'jupyterlab', 'latex', 'multi-language-demo', 'appmode', 'apt_install', 'data-quilt', 'demo-julia', 'getting-data', 'getting-data-xroot', 'julia-python', 'jupyter-sos', 'minimal-dockerfile', 'nix', 'python-conda_pip', 'r_with_python', 'requirements', 'setup.py', 'stencila-r'],
        type: 'nominal'
      }),
      y: Y({
        field: 'pony',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Organization: binder-examples'
  }), Chart({
    data:         org                repo  pony
    30  ipython    ipython-in-depth     4
    33  ipython    ipython_genutils     4
    28  ipython             ipython     3
    31  ipython     ipython-website     3
    35  ipython           traitlets     3
    25  ipython           ipykernel     2
    26  ipython               ipynb     2
    34  ipython           rlipython     2
    23  ipython                disp     1
    24  ipython       front-to-back     1
    27  ipython         ipyparallel     1
    29  ipython         ipython-doc     1
    32  ipython  ipython.github.com     1
    36  ipython         usersurveys     1
    37  ipython           xkcd-font     1,
    encoding: FacetedEncoding({
      color: Color({
        field: 'org',
        type: 'nominal'
      }),
      x: X({
        field: 'repo',
        sort: ['ipython-in-depth', 'ipython_genutils', 'ipython', 'ipython-website', 'traitlets', 'ipykernel', 'ipynb', 'rlipython', 'disp', 'front-to-back', 'ipyparallel', 'ipython-doc', 'ipython.github.com', 'usersurveys', 'xkcd-font'],
        type: 'nominal'
      }),
      y: Y({
        field: 'pony',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Organization: ipython'
  }), Chart({
    data:          org                        repo  pony
    47   jupyter               docker-stacks     7
    54   jupyter                     jupyter     7
    91   jupyter                    notebook     7
    49   jupyter       enhancement-proposals     6
    95   jupyter                   qtconsole     6
    64   jupyter           jupyter.github.io     5
    81   jupyter                   nbconvert     5
    107  jupyter                    testpath     5
    41   jupyter                colaboratory     4
    68   jupyter         jupyter_kernel_test     4
    74   jupyter        kernel_gateway_demos     4
    84   jupyter                    nbformat     4
    97   jupyter                     roadmap     4
    106  jupyter                   terminado     4
    45   jupyter

In [9]:
prs['createdAt'] = pd.to_datetime(prs['createdAt'])
author_count_by_month = (
    prs.groupby(['org', 'author'])
    .resample('30D', on='createdAt')
    .agg({'id': 'count'})
    .rename(columns={'id': 'count'})
)

In [10]:
charts = []
for org, iauthors in top_authors.groupby('org'):
    author_order = iauthors.sort_values(['count'], ascending=False).index.values.tolist()
    ch = alt.Chart(iauthors.reset_index(), title=org).mark_bar().encode(
        x=alt.X('author', sort=author_order),
        y='count',
    )
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:                  author  count              org
    0               betatim      2  binder-examples
    1                kannes      6  binder-examples
    2             emigre459      2  binder-examples
    3           bitterspice      2  binder-examples
    4   3keepmovingforward3      2  binder-examples
    5                 vsoch      8  binder-examples
    6               betatim      8  binder-examples
    7               davidar      4  binder-examples
    8           matheusmota      4  binder-examples
    9          davidanthoff      2  binder-examples
    10             cboettig      2  binder-examples
    11              Carreau      2  binder-examples
    12               akarve      2  binder-examples
    13      matthewfeickert      2  binder-examples
    14              Carreau      2  binder-examples
    15               BoPeng      2  binder-examples
    16              parente      2  binder-examples
    17              betatim      2  binder-examples
    18            choldgraf      2  binder-examples
    19              betatim      2  binder-examples
    20            yuvipanda      2  binder-examples
    21              jzf2101      2  binder-examples
    22            bsyouness      2  binder-examples
    23              betatim      2  binder-examples
    24           jamesmyatt      2  binder-examples
    25            choldgraf      2  binder-examples
    26              betatim      2  binder-examples
    27              betatim      2  binder-examples
    28            fomightez      2  binder-examples
    29            yuvipanda      4  binder-examples
    30         RaoOfPhysics      4  binder-examples
    31             coatless      2  binder-examples
    32            choldgraf      2  binder-examples
    33              betatim      2  binder-examples
    34             Fusiakja      2  binder-examples
    35              sebkopf      2  binder-examples
    36              betatim      4  binder-examples
    37              cranmer      2  binder-examples
    38           benmarwick      2  binder-examples,
    encoding: FacetedEncoding({
      x: X({
        field: 'author',
        sort: ['vsoch', 'betatim', 'kannes', 'yuvipanda', 'betatim', 'davidar', 'matheusmota', 'RaoOfPhysics', 'betatim', 'jamesmyatt', 'choldgraf', 'betatim', 'betatim', 'fomightez', 'choldgraf', 'coatless', 'bsyouness', 'betatim', 'Fusiakja', 'sebkopf', 'cranmer', 'betatim', 'betatim', 'jzf2101', 'yuvipanda', 'choldgraf', 'betatim', 'parente', 'BoPeng', 'Carreau', 'matthewfeickert', 'akarve', 'Carreau', 'cboettig', 'davidanthoff', '3keepmovingforward3', 'bitterspice', 'emigre459', 'benmarwick'],
        type: 'nominal'
      }),
      y: Y({
        field: 'count',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'binder-examples'
  }), Chart({
    data:               author  count      org
    0             rgbkrk      6  ipython
    1            Carreau      3  ipython
    2             tgb417      3  ipython
    3              minrk    291  ipython
    4            Carreau     84  ipython
    5          takluyver     57  ipython
    6      SylvainCorlay     18  ipython
    7         jasongrout     15  ipython
    8           dwillmer     15  ipython
    9           jdfreder      9  ipython
    10         blink1073      9  ipython
    11            rgbkrk      6  ipython
    12            ivanov      6  ipython
    13           hroncok      6  ipython
    14      jenshnielsen      6  ipython
    15          itziakos      6  ipython
    16        bergtholdt      6  ipython
    17            aburgm      6  ipython
    18           cHYzZQo      6  ipython
    19        ccordoba12      3  ipython
    20            fgregg      3  ipython
    21     AhmedRedaAmin      3  ipython
    22        ian-r-rose      3  ipython
    23           Carreau     30  ipython
    24         yuvipanda     15  ipython
    25       blairhudson      6  ipython
    26       to

In [11]:
%%html
<script src="https://cdn.rawgit.com/parente/4c3e6936d0d7a46fd071/raw/65b816fb9bdd3c28b4ddf3af602bfd6015486383/code_toggle.js"></script>